# NF26/AI07 - TD4 : Création d'un tableau de bord



In [2]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [4]:
!pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 56.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
blosc2 3.2.0 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is 

In [20]:
import pandas as pd
import numpy as np
from datetime import datetime, date

import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
# Choix du fuseau horaire (Solution trouvée par Thomas F.) :
spark.conf.set("spark.sql.session.timeZone", "UTC")

## Introduction

L'objectif de ce TD est d'implémenter un tableau de bord à partir d'un modèle Étoile alimenté par 6 différentes bases de données transactionnelles non-normailisées pour procéder des requêtes visant à avoir une aperçu sur les données d'un Hôpital. Les 6 diffrérentes bases de données transactionnelles données sont :
- `data_administration`: Ensemble des données administratives liées à chaque consultation.
- `data_medecins`: Ensemble des données liées aux informations associées à chaque Médecin.
- `data_diagnostics`: Ensemble des données liées aux diagnostic de chaque patient lors de chaque consultation.
- `data_treatments`: Ensemble des données liées aux descriptions de chaque traitement.
- `data_medicaments`: Ensemble des informations caractérisant chaque Médicament.
- `data_chambres`: Ensemble des informations caractérisant chaque Chambre de l'Hôpital.

On souhaite notamment pouvoir illustrer les requêtes suivantes :
- "*Quel a été l'âge moyen des patients qui ont eu une certaine Pathology durant une certaine période ?*".
- "*Quel Médicament a été le plus prescrit (en terme de quantité) pour une certaine Pathology durant une certaine période ?*".
- "*Combien de chambres ont accueilli des patients diagnostiqués d'une certaine Pathology durant une certaine période ?*".
- "*Quelle est la proportion de medecins (par spécialité) qui ont dignostiqué une certaine `Pathology` durant une certaine période ?*".


## Lecture des données



Proposition de Code par Arman S-M afin de charger un dossier facilement sur Google Colab :

In [ ]:
# Proposition de Code par Arman S-M
import zipfile

def unzip_data(filename):
    #Unzips filename into the current working directory.
    #Args:
    #filename (str): a filepath to a target zip folder to be unzipped.
    zip_ref = zipfile.ZipFile(filename, "r")
    zip_ref.extractall()
    zip_ref.close()

# Usage : unzip_data("nf26-td2.zip")

Les cellules ci-dessous permettent de lire les différentes bases de données transactionnelles en format `pyspark.pandas.frame.DataFrame`.

In [6]:
psdf_administration = ps.read_csv('/content/drive/MyDrive/Colab Notebooks/TD4/data_administration.csv', index_col='KeyConsult')
psdf_administration.head(10)

,KeyPatient,Name,FirstName,NumSecu,Date_In,Pathology,KeyChambre,ChambreNumber,Date_Out
KeyConsult,,,,,,,,,
KeyConsult45056,KeyPatient10546,Name10546,FistName10546,NS45057,2023-11-24 23:00:00,Pathology22,KeyChambre81,C081,2023-11-25 23:00:00
KeyConsult45057,KeyPatient45482,Name45482,FistName45482,NS45058,2023-11-24 23:00:00,Pathology34,KeyChambre102,C102,2023-11-25 23:00:00
KeyConsult45058,KeyPatient37050,Name37050,FistName37050,NS45059,2023-11-24 23:00:00,Pathology84,KeyChambre22,C022,2023-12-04 23:00:00
KeyConsult45059,KeyPatient39793,Name39793,FistName39793,NS45060,2023-11-24 23:00:00,Pathology78,KeyChambre4,C004,2023-11-30 23:00:00
KeyConsult45060,KeyPatient17652,Name17652,FistName17652,NS45061,2023-11-24 23:00:00,Pathology15,KeyChambre29,C029,2023-11-25 23:00:00
KeyConsult45061,KeyPatient40034,Name40034,FistName40034,NS45062,2023-11-24 23:00:00,Pathology7,KeyChambre60,C060,2023-11-25 23:00:00
KeyConsult45062,KeyPatient38237,Name38237,FistName38237,NS45063,2023-11-24 23:00:00,Pathology7,KeyChambre5,C005,2023-11-26 23:00:00
KeyConsult45063,KeyPatient17519,Name17519,FistName17519,NS45064,2023-11-24 23:00:00,Pathology67,KeyChambre84,C084,2023-12-02 23:00:00
KeyConsult45064,KeyPatient28813,Name28813,FistName28813,NS45065,2023-11-24 23:00:00,Pathology56,KeyChambre51,C051,2023-11-25 23:00:00


In [7]:
psdf_medecins = ps.read_csv('/content/drive/MyDrive/Colab Notebooks/TD4/data_medecins.csv', index_col='KeyMedecin')
psdf_medecins.head(10)

,NameMedecin,FirstNameMedecin,SpecialityMedecin
KeyMedecin,,,
keyMedecin26,NameMedecin26,FirstNameMedecin26,Generaliste
keyMedecin27,NameMedecin27,FirstNameMedecin27,Chirurgien
keyMedecin28,NameMedecin28,FirstNameMedecin28,Dermatologiste
keyMedecin29,NameMedecin29,FirstNameMedecin29,Generaliste
keyMedecin16,NameMedecin16,FirstNameMedecin16,Chirurgien
keyMedecin17,NameMedecin17,FirstNameMedecin17,Chirurgien
keyMedecin18,NameMedecin18,FirstNameMedecin18,Chirurgien
keyMedecin19,NameMedecin19,FirstNameMedecin19,Generaliste
keyMedecin6,NameMedecin6,FirstNameMedecin6,Dermatologiste


In [8]:
psdf_diagnostics = ps.read_csv('/content/drive/MyDrive/Colab Notebooks/TD4/data_diagnostics.csv', index_col='KeyConsult')
psdf_diagnostics.head(10)

,KeyMedecin,KeyPatient,NamePatient,FirstNamePatient,NumSecu,Age,Weight,Temperature,Tension,Diabete,Pathology,KeyTreatment
KeyConsult,,,,,,,,,,,,
KeyConsult45056,keyMedecin26,KeyPatient10546,Name10546,FistName10546,NS45057,24,58.759522,37.459899,135.700753,0,Pathology22,KeyTreatment45056
KeyConsult45057,keyMedecin22,KeyPatient45482,Name45482,FistName45482,NS45058,19,58.422513,37.488268,95.790925,0,Pathology34,KeyTreatment45057
KeyConsult45058,keyMedecin14,KeyPatient37050,Name37050,FistName37050,NS45059,50,80.125748,38.023957,89.173819,1,Pathology84,KeyTreatment45058
KeyConsult45059,keyMedecin29,KeyPatient39793,Name39793,FistName39793,NS45060,76,69.306556,39.966692,116.606928,0,Pathology78,KeyTreatment45059
KeyConsult45060,keyMedecin18,KeyPatient17652,Name17652,FistName17652,NS45061,22,56.173338,37.463554,161.955995,0,Pathology15,KeyTreatment45060
KeyConsult45061,keyMedecin21,KeyPatient40034,Name40034,FistName40034,NS45062,16,55.421938,37.554036,153.418985,0,Pathology7,KeyTreatment45061
KeyConsult45062,keyMedecin18,KeyPatient38237,Name38237,FistName38237,NS45063,22,63.067835,37.213361,171.359467,0,Pathology7,KeyTreatment45062
KeyConsult45063,keyMedecin2,KeyPatient17519,Name17519,FistName17519,NS45064,34,58.652805,38.459865,107.910361,0,Pathology67,KeyTreatment45063
KeyConsult45064,keyMedecin21,KeyPatient28813,Name28813,FistName28813,NS45065,27,59.233520,37.535722,123.634315,0,Pathology56,KeyTreatment45064


In [9]:
psdf_treatments = ps.read_csv('/content/drive/MyDrive/Colab Notebooks/TD4/data_treatments.csv', index_col='KeyTreatment')
psdf_treatments.head(10)

,KeyMedicament,QuantityMedicament
KeyTreatment,,
KeyTreatment20495,KeyMed513,1
KeyTreatment20495,KeyMed852,2
KeyTreatment20495,KeyMed265,1
KeyTreatment20496,KeyMed82,2
KeyTreatment20497,KeyMed67,1
KeyTreatment20497,KeyMed21,3
KeyTreatment20498,KeyMed697,1
KeyTreatment20499,KeyMed2,2
KeyTreatment20500,KeyMed47,2


In [10]:
psdf_medicaments = ps.read_csv('/content/drive/MyDrive/Colab Notebooks/TD4/data_medicaments.csv', index_col='KeyMedicament')
psdf_medicaments.head(10)

,NameMedicament
KeyMedicament,
KeyMed913,Medicament913
KeyMed914,Medicament914
KeyMed915,Medicament915
KeyMed916,Medicament916
KeyMed917,Medicament917
KeyMed918,Medicament918
KeyMed919,Medicament919
KeyMed920,Medicament920
KeyMed921,Medicament921


In [11]:
psdf_chambres = ps.read_csv('/content/drive/MyDrive/Colab Notebooks/TD4/data_chambres.csv', index_col='KeyChambre')
psdf_chambres.head(10)

,ChambreNumber
KeyChambre,
KeyChambre89,C089
KeyChambre90,C090
KeyChambre91,C091
KeyChambre92,C092
KeyChambre93,C093
KeyChambre94,C094
KeyChambre95,C095
KeyChambre96,C096
KeyChambre97,C097


Afin de pouvoir réaliser notre objectif, nous pourrons convertir ces données en format `pyspark.sql.dataframe.DataFrame`.

In [12]:
sdf_administration = psdf_administration.to_spark(index_col='KeyConsult')
sdf_medecins = psdf_medecins.to_spark(index_col='KeyMedecin')
sdf_diagnostics = psdf_diagnostics.to_spark(index_col='KeyConsult')
sdf_treatments = psdf_treatments.to_spark(index_col='KeyTreatment')
sdf_medicaments = psdf_medicaments.to_spark(index_col='KeyMedicament')
sdf_chambres = psdf_chambres.to_spark(index_col='KeyChambre')

In [13]:
from pyspark.sql.functions import *

## 1. Création et alimentation du Modèle étoile

Tout comme dans les TD2 et TD3, nous allons créer un modèle en étoile permettant de procéder facilement les requêtes suivantes :
- "*Quel a été l'âge moyen des patients qui ont eu une certaine Pathology durant une certaine période ?*".
- "*Quel Médicament a été le plus prescrit (en terme de quantité) pour une certaine Pathology durant une certaine période ?*".
- "*Combien de chambres ont des patients diagnostiqués d'une certaine Pathology durant une certaine période ?*".
- "*Quelle est la proportion de medecins (par spécialité) qui ont dignostiqué une certaine `Pathology` durant une certaine période ?*".

**Question 1.** À partir des 6 bases de données transactionnelles, créer le modèle étoile comme illustré en pièce jointe (similaire aux TD2 et TD3)

In [25]:
t1 = sdf_administration.join(sdf_diagnostics, "KeyConsult")
facts = t1.select("*").withColumn("KeyDates", monotonically_increasing_id())
facts = facts.select("KeyConsult", sdf_administration.KeyPatient, "KeyMedecin", "KeyTreatment", "KeyDates", "KeyChambre")

dim_patients = sdf_diagnostics.select("KeyPatient", "KeyConsult", "NamePatient", "FirstNamePatient", "NumSecu", "Age", "Weight", "Temperature", "Tension", "Diabete", "Pathology")

dates = sdf_administration.select("Date_In", "Date_Out")
dim_dates = dates.select("*").withColumn("KeyDates", monotonically_increasing_id()).select("KeyDates", "Date_In", "Date_Out")

dim_medecin = sdf_medecins.select("*")

dim_traitement = sdf_treatments.join(sdf_medicaments, "KeyMedicament").select("KeyTreatment", "KeyMedicament", "QuantityMedicament", "NameMedicament")

dim_chambre = sdf_chambres.select("*")

facts.show()

+---------------+---------------+------------+-----------------+--------+-------------+
|     KeyConsult|     KeyPatient|  KeyMedecin|     KeyTreatment|KeyDates|   KeyChambre|
+---------------+---------------+------------+-----------------+--------+-------------+
|KeyConsult45056|KeyPatient10546|keyMedecin26|KeyTreatment45056|       0| KeyChambre81|
|KeyConsult45057|KeyPatient45482|keyMedecin22|KeyTreatment45057|       1|KeyChambre102|
|KeyConsult45058|KeyPatient37050|keyMedecin14|KeyTreatment45058|       2| KeyChambre22|
|KeyConsult45059|KeyPatient39793|keyMedecin29|KeyTreatment45059|       3|  KeyChambre4|
|KeyConsult45060|KeyPatient17652|keyMedecin18|KeyTreatment45060|       4| KeyChambre29|
|KeyConsult45061|KeyPatient40034|keyMedecin21|KeyTreatment45061|       5| KeyChambre60|
|KeyConsult45062|KeyPatient38237|keyMedecin18|KeyTreatment45062|       6|  KeyChambre5|
|KeyConsult45063|KeyPatient17519| keyMedecin2|KeyTreatment45063|       7| KeyChambre84|
|KeyConsult45064|KeyPatient28813

In [42]:
facts.write.csv('facts.csv', mode="overwrite", header=True)
dim_patients.write.csv('dim_patients.csv', mode="overwrite", header=True)
dim_dates.write.csv('dim_dates.csv', mode="overwrite", header=True)
dim_medecin.write.csv('dim_medecin.csv', mode="overwrite", header=True)
dim_traitement.write.csv('dim_traitement.csv', mode="overwrite", header=True)
dim_chambre.write.csv('dim_chambre.csv', mode="overwrite", header=True)

## 2. Création d'un tableau de bord

Les tableaux de bord sont très utiles dans le Data-Warehouse pour répondre et illustrer des requêtes SQL fréquentes.
Pour ce faire, nous allons utiliser `Streamlit` : https://streamlit.io/

**Question.** À partir du modèle étoile précédement construit, créer un tableau de bord en utilisant Streamlit pour répondre et illustrer les requêtes suivantes :

- "*Quel a été l'âge moyen des patients qui ont eu une certaine Pathology durant une certaine période ?*".
- "*Quel Médicament a été le plus prescrit (en terme de quantité) pour une certaine Pathology durant une certaine période ?*".
- "*Combien de chambres ont des patients diagnostiqués d'une certaine Pathology durant une certaine période ?*".
- "*Quelle est la proportion de medecins (par spécialité) qui ont dignostiqué une certaine `Pathology` durant une certaine période ?*".

In [15]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00


In [16]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
added 22 packages in 4s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [74]:
%%writefile app.py

import streamlit as st
import pandas as pd
import numpy as np
from datetime import datetime, date

import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import Row

from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.session.timeZone", "UTC")

facts = ps.read_csv('/content/facts.csv').to_spark()
dim_chambre = ps.read_csv('/content/dim_chambre.csv').to_spark()
dim_dates = ps.read_csv('/content/dim_dates.csv').to_spark()
dim_medecin = ps.read_csv('/content/dim_medecin.csv').to_spark()
dim_patients = ps.read_csv('/content/dim_patients.csv').to_spark()
dim_traitement = ps.read_csv('/content/dim_traitement.csv').to_spark()


st.title("TD4 - Tableau de bord")


# -----------------------------------------------------------------
st.header("Question 1")

pathologies = dim_patients.select('Pathology').distinct().toPandas()['Pathology'].tolist()

selected_pathology = st.selectbox('Select Pathology', pathologies)
age = dim_patients.filter(col("Pathology") == selected_pathology).agg(avg(col("Age"))).collect()[0][0]
st.metric("Age moyen :", age)

# -----------------------------------------------------------------
st.header("Question 2")

selected_pathology_2 = st.selectbox('Select Pathology 2', pathologies)
start_date = st.date_input("Date de début")
end_date = st.date_input("Date de fin")


t3 = dim_traitement.join(facts, "KeyTreatment").join(dim_patients, "KeyPatient").join(dim_dates, "KeyDates")
t3 = t3.filter(col("Pathology") == selected_pathology_2).filter((col("Date_In") >= start_date) & (col("Date_Out") <= end_date)).select("KeyMedicament", "QuantityMedicament")
t3_total = t3.groupBy("KeyMedicament").agg(sum("QuantityMedicament").alias("TotalQuantity")).orderBy("TotalQuantity", ascending=False).collect()[0]
st.metric("Médicament :", t3_total[0])
st.metric("Quentité :", t3_total[1])




Overwriting app.py


In [75]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl https://loca.lt/mytunnelpassword

34.125.163.176⠙your url is: https://fancy-files-chew.loca.lt
